In [ ]:
!pip install -q streamlit

In [ ]:
pip install streamlit-mic-recorder

In [ ]:
from google.colab import drive
drive.mount('/content/drive')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%writefile app.py

import streamlit as st
import librosa
import numpy as np
import pandas as pd
import os
from tensorflow.keras.models import load_model as model_loader
import joblib
from streamlit_mic_recorder import mic_recorder
import wave


audio_file = None
model = None

def load_model():
    try:
        model = model_loader("/content/drive/MyDrive/CRNNSpoof_Model.h5")
        return model
    except FileNotFoundError:
        st.sidebar.error("Model file not found. Please make sure the file exists.")
        return None

model = load_model()
st.sidebar.success("Model loaded!")


st.title("Voice authentication 🎵:")

audio_file = st.file_uploader("Upload Audio", type=["wav","mp3","flac"])

#start test


def callback():
    if st.session_state.my_recorder_output:
        audio_bytes = st.session_state.my_recorder_output['bytes']
        st.audio(audio_bytes)




audio_file_test = mic_recorder(key='my_recorder', callback=callback, format = 'wav')
if audio_file_test is not None:
    with wave.open('output.wav', 'wb') as wav_file:
            audio_bytes = st.session_state.my_recorder_output['bytes']
            wav_file.setnchannels(1) # Mono
            wav_file.setsampwidth(2) # Sample width in bytes
            wav_file.setframerate(44100) # Sample rate
            wav_file.writeframes(audio_bytes)




#end test
st.sidebar.header("Play the audio")
st.sidebar.audio(audio_file)


# Function to extract features from an audio file
def extract_features(file_path, segment_length):
    try:
        # Load the audio file
        y, sr = librosa.load(file_path)
        # Calculate the number of segments based on the segment length and audio length
        num_segments = int(np.ceil(len(y) / float(segment_length * sr)))
        print("num_segments: "+ str(num_segments))
        # Initialize a list to store the features for this file
        features = []

        # Extract features for each segment
        for i in range(num_segments):
            # Calculate start and end frame for the current segment
            start_frame = i * segment_length * sr
            end_frame = min(len(y), (i + 1) * segment_length * sr)
            # Extract audio for this segment
            y_segment = y[start_frame:end_frame]

            # Extract features
            chroma_stft = np.mean(librosa.feature.chroma_stft(y=y_segment, sr=sr))
            rms = np.mean(librosa.feature.rms(y=y_segment))
            spec_cent = np.mean(librosa.feature.spectral_centroid(y=y_segment, sr=sr))
            spec_bw = np.mean(librosa.feature.spectral_bandwidth(y=y_segment, sr=sr))
            rolloff = np.mean(librosa.feature.spectral_rolloff(y=y_segment, sr=sr))
            zcr = np.mean(librosa.feature.zero_crossing_rate(y_segment))
            mfccs = librosa.feature.mfcc(y=y_segment, sr=sr)
            mfccs_mean = np.mean(mfccs, axis=1)

            # Append the extracted features to the list
            features.append([chroma_stft, rms, spec_cent, spec_bw, rolloff, zcr, *mfccs_mean])

        return np.array(features)
    except Exception as e:
        print(f"Error processing {file_path}: {e}")
        return None


def make_prediction(features):
    output = model.predict(features)
    return [1 if value >= 0.5 else 0 for value in output]


if st.sidebar.button("Test Audio"):
  if audio_file is not None or os.path.isfile("./output.wav"):
    if os.path.isfile("./output.wav"):
      audio_file = "./output.wav"
    st.sidebar.success("Processing audio...")

    features = extract_features(audio_file, segment_length=1)
    X = pd.DataFrame(features, columns = ['chroma_stft', 'rms', 'spectral_centroid', 'spectral_bandwidth', 'rolloff', 'zero_crossing_rate', 'mfcc1', 'mfcc2', 'mfcc3', 'mfcc4', 'mfcc5', 'mfcc6', 'mfcc7', 'mfcc8', 'mfcc9', 'mfcc10', 'mfcc11', 'mfcc12', 'mfcc13', 'mfcc14', 'mfcc15', 'mfcc16', 'mfcc17', 'mfcc18', 'mfcc19', 'mfcc20'])
    scaler = joblib.load("/content/drive/MyDrive/CRNNSpoof_scale.joblib")

    X_test_scaled = scaler.transform(features.reshape(features.shape[0], -1))
    X_test_scaled = X_test_scaled.reshape(features.shape)

    X_test_scaled = X_test_scaled.reshape(X_test_scaled.shape[0], X_test_scaled.shape[1], 1)


    # Use the model to make predictions on the dataset
    st.write("Making predictions with the model...")

    if features is not None:
        prediction_result = make_prediction(X_test_scaled)

    if np.average(prediction_result) <= 0.4:
      st.write("Prediction: Fake ")
    else:
      st.write("Prediction: Real ")
    if os.path.isfile("./output.wav"):
      os.remove("./output.wav")
  else:
    st.sidebar.warning("No Model")








Overwriting app.py


In [ ]:
!npm install localtunnel

In [ ]:
import urllib
print("Password/Enpoint IP for localtunnel is:",urllib.request.urlopen('https://ipv4.icanhazip.com').read().decode('utf8').strip("\n"))

In [ ]:
!streamlit run app.py &>/content/logs.txt &

In [ ]:
!npx localtunnel --port 8501